# **Introdução**

O objetivo desse trabalho é implementar "Automated Bias Discover Task" descrito em: _Discover the Unknown Biased Attribute of an Image Classifier
Zhiheng Li, Chenliang Xu_

O workflow descrito no paper, pode ser bem resumido pela seguinte imagem:

# ![title](images/workflow.png)

Percaba que primeiramente precisamos obter um hiperplano do atributo enviesado e, depois disso, a tarefa de encontrar o viés do classificador continua.
Nesse sentido, o escopo desse trabalho é, primeiramente obter esse hiperplano otimizado.

Para fazer isso, precisamos de duas coisas:
1. Um classificador enviesado ✅
2. Um modelo gerativo capaz de reproduzir a distribuição de probabilidades do dataset desse classificador ✅

Como já possuímos esses dois pre-requisitos, podemos implementar esse workflow. É importante perceber que faremos uma abordagem 1 vs all para cada classe no problema. Podendo encontrar diversos vieses presentes nesse dataset, um para cada classe.

# **Step 1: Hyperplane optimization**

#### **A implementação da task de treinamento pode ser encontrada em ``bias_discover_training_task.py``**

# **Step 2: Human interpretation**

### **1. Fazer os imports necessários.**

In [2]:
from modules.discoverer import GenerativeModel, BiasDiscoverer, one_vs_all_inference
from modules.model_loader import load_gen_model, load_classifier
import torch
import matplotlib.pyplot as plt

### **2. Definir constantes.**

In [ ]:
GENERATOR_PATH = "../StyleGAN/trained_models/netG.pth"
MAPPING_NETWORK_PATH = "../StyleGAN/trained_models/mappingNetwork.pth"
CLASSIFIER_PATH = "../../../Classifier/models/model.pth"

Z_DIM = 256
EPOCHS = 1
BATCH_SIZE = 32
DEVICE = "cpu"
TARGET_CLASS = 1

### **3. Instanciar modelos.**

In [ ]:
gen_model = GenerativeModel(load_gen_model(GENERATOR_PATH, MAPPING_NETWORK_PATH, DEVICE), mapping_network)
biased_classifier = load_classifier(CLASSIFIER_PATH, DEVICE)
bias_discoverer = BiasDiscoverer(Z_DIM, gen_model, biased_classifier)

### **4. Gerar as traversal images.**

In [ ]:
z_data_points = torch.rand(BATCH_SIZE, Z_DIM)
latent_codes = bias_discoverer.generate_latent_codes(z_data_points)
traversal_images = generate_traversal_images(gen_model, latent_codes)
probs_predictions = one_vs_all_inference(biased_classifier, traversal_images, TARGET_CLASS)

### **5. Plotar as traversal images com seus valores preditos.**

In [ ]:
def display_images_with_scores(images, scores):
    fig, ax = plt.subplots(1, len(images), figsize=(15, 5))
    
    for i, (img, score) in enumerate(zip(images, scores)):
        # Display image
        ax[i].imshow(img)
        ax[i].axis('off')  # hide axes
        
        # Annotate the score. Adjust (x,y) values as per your needs.
        ax[i].annotate(f"{score:.2f}", (0, img.shape[0] + 15), color="black", weight="bold", fontsize=12, ha='left')
    
    plt.tight_layout()
    plt.show()

In [ ]:
for i in range(BATCH_SIZE):
    display_images_with_scores(traversal_images[i], probs_predictions[i])